In [14]:
!pip install tpot
from tpot import TPOTClassifier
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import KFold
from sklearn import tree

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [15]:
df = pd.read_csv('/content/gdrive/MyDrive/Bisnis_Cerdas/Tugas4/Blooddonorprediction/dataset.csv', na_values = {'?'})
df.head(10)

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0
5,4,4,1000,4,0
6,2,7,1750,14,1
7,1,12,3000,35,0
8,2,9,2250,22,1
9,5,46,11500,98,1


In [16]:
X = []
Y = []
list_clf = []

In [17]:
df = df.values

In [18]:
X = df[:, :(df.shape[1]-1)]
Y = df[:, df.shape[1]-1]

In [19]:
tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2)
tpot.fit(X, Y)

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7927874720357941

Generation 2 - Current best internal CV score: 0.7927874720357941

Generation 3 - Current best internal CV score: 0.7927874720357941

Generation 4 - Current best internal CV score: 0.7927874720357941

Generation 5 - Current best internal CV score: 0.7927874720357941

Best pipeline: LinearSVC(input_matrix, C=10.0, dual=False, loss=squared_hinge, penalty=l2, tol=0.01)


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=5,
               log_file=None, max_eval_time_mins=5, max_time_mins=None,
               memory=None, mutation_rate=0.9, n_jobs=1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=50,
               random_state=None, scoring=None, subsample=1.0, template=None,
               use_dask=False, verbosity=2, warm_start=False)

In [20]:
tpot.fitted_pipeline_

Pipeline(memory=None,
         steps=[('linearsvc',
                 LinearSVC(C=10.0, class_weight=None, dual=False,
                           fit_intercept=True, intercept_scaling=1,
                           loss='squared_hinge', max_iter=1000,
                           multi_class='ovr', penalty='l2', random_state=None,
                           tol=0.01, verbose=0))],
         verbose=False)

In [21]:
clf_tree = tree.DecisionTreeClassifier()
clf_svm = SVC()
clf_perceptron = Perceptron()
clf_KNN = KNeighborsClassifier()
clf_nb = BernoulliNB()

In [22]:
list_clf = [clf_tree, clf_svm, clf_perceptron, clf_KNN, clf_nb, tpot.fitted_pipeline_]

In [23]:
kf = KFold(n_splits=5)
kf.get_n_splits(X)
c = 1
for clfs in list_clf:
    print(c)
    c += 1
    a = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        clfs.fit(X_test, Y_test)  
        print(clfs.score(X_train, Y_train)) 
        a += clfs.score(X_train, Y_train)
    a = a/5
    print("Average=",a,"\n")
    print(clfs,"\n")

1
0.7073578595317725
0.6956521739130435
0.6321070234113713
0.7128547579298832
0.6844741235392321
Average= 0.6864891876650605 

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best') 

2
0.81438127090301
0.7558528428093646
0.7290969899665551
0.7679465776293823
0.7262103505843072
Average= 0.7586976063785238 

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False) 

3
0.18561872909698995
0.7558528428093646
0.7290969899665551
0.78464